### Tracks is the big database of songs that we recommend from. StreamingHistory is Nathan's streaming history. We'll use Streaming History to predict Nathan's music tastes.

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
#from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score
from sklearn.linear_model import LogisticRegression

In [13]:
def clean_and_merge_data(song_data, user_data):
    # Read in tracks.csv
    music = song_data

    # Data Cleaning Steps
    music = music.drop_duplicates(subset = ['name', 'artists'])
    user_grouped = user_data.groupby(by = 'trackName').max().reset_index()
    music['primary_artist'] = music['artists'].str.strip('[]').str.split(',').str[0].str.strip('\'\'')
    merged = pd.merge(user_grouped, music, left_on = ['trackName', 'artistName'], right_on = ['name', 'primary_artist'])
    merged.loc[merged['msPlayed'] <= 30000, 'Likes_Song'] = 0
    merged.loc[merged['msPlayed'] > 30000, 'Likes_Song'] = 1
    
    return merged

def split_data(song_data, user_data):
    
    merged_data = clean_and_merge_data(song_data, user_data)
    
    # This grabs the following features: ['danceability', 'energy', 'key', 'loudness', 'mode', 
    # 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']
    filters = list(merged_data.columns[12:24])
    train = merged_data[filters + ["Likes_Song"]]
    user_x = train[filters]
    user_y = train["Likes_Song"]
    
    return user_x, user_y, filters

def RandomForestModel(song_data, user_data):
    
    # Split data into train x and train y
    user_x, user_y, filters = split_data(song_data, user_data)
    
    # Make test data stuff
    test = song_data[filters]
    name = song_data['name']
    artist = song_data['artists'].str.strip('[]').str.split(',').str[0].str.strip('\'\'')
    
    model = RandomForestClassifier(n_estimators = 70, criterion = "entropy", max_depth = 7, random_state = 123)
    model.fit(user_x, user_y)
    predictions = model.predict(test)

    test['recommendations'] = predictions
    test['name'] = name
    test['artist'] = artist
    
    recommendations = test[test['recommendations'] == 1].sample(50)[['name','artist']]
    
    return recommendations

In [14]:
tracks = pd.read_csv("tracks.csv")
nathan = pd.read_csv("StreamingHistory.csv")
RandomForestModel(tracks, nathan)

C:\Users\natha\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\natha\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\natha\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#inde

,name,artist
163616,"1962 / Oodi ""Varjoille""",Hector
550691,裸足のまんまで,Tsuyoshi Nagabuchi
22610,Je te veux,Erik Satie
460391,Maintenant ou jamais (feat. X-Men),Fonky Family
562106,ビンテージ,Official HIGE DANdism
473683,Espinas en el Rosal,Tambó Tambó
227954,Припевочка,Zemfira
253638,Fuente de Energia,Estopa
351600,Me enamoras con todo,Café Quijano
339292,Irena's Theme,Giorgio Moroder


In [65]:
#
# Gave up on 29/11/22 at 12pm. idk how to fix the nans bro.
# I thought the nans were caused by the values in "y" being floats
# instead of integers, but when I changed the nums to integers then
# the type changed to a numpy array instead of a Panda series.
#
def grid_search(song_data, user_data):
    x, y, filters = split_data(song_data, user_data)
    print("This is y: ", type(y))
    print("This is y_values: ", y.values)
    #y = y.values.astype(int)
    print('This is int_y_values: ', type(y))
    x_train, y_train, x_test, y_test = train_test_split(x, y, test_size = 0.5, random_state = 123)
#     x_test = x_test(lambda x: int(x))
#     y_test = y_test(lambda y: int(y))
    print("x_train: ", x_train)
    print("y_train: ", y_train)
    print("x_test: ", x_test)
    print("y_test: ", y_test)
    n_estimators = [int(x) for x in np.linspace(start = 10, stop = 80, num = 10)]
    max_features = ["auto", "sqrt"]
    max_depth = [2, 6]
    min_samples_split = [2, 5]
    min_samples_leaf = [1, 2]
    bootstrap = [True, False]
    
    param_grid = {"n_estimators": n_estimators, "max_features": max_features,
                  "max_depth": max_depth, "min_samples_split": min_samples_split,
                  "min_samples_leaf": min_samples_leaf, "bootstrap": bootstrap}
    
    rf_model = RandomForestClassifier()
    rf_grid = GridSearchCV(estimator = rf_model, param_grid = param_grid, cv = 3, verbose = 2, n_jobs = 4)
    
    rf_grid.fit(x_train, y_train)
    
    return rf_grid.best_params_

In [66]:
# I know I'm passing in the right csvs
grid_search(tracks, nathan)

C:\Users\natha\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


This is y:  <class 'pandas.core.series.Series'>
This is y_values:  [1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
This is int_y_values:  <class 'pandas.core.series.Series'>
x_train:      danceability  energy  key  loudness  mode  speechiness  acousticness  \
32         0.615   0.711   11    -5.507     1       0.0779      0.044400   
14         0.458   0.561   10    -4.819     1       0.0269      0.096500   
23         0.428   0.276    4   -11.729     1       0.0312      0.732000   
9          0.505   0.710    3    -3.015     1       0.0321      0.190000   
15         0.434   0.944    5    -2.641     1       0.0581      0.076900   
29         0.534   0.878    4    -5.134     1       0.0934      0.005030   
0          0.627   0.810    2    -5.296     0       0.0431      0.012500   
1          0.422   0.264    8    -7.064     1       0.0322      0.922000   
22         0.545   0.783    5    -4.261     0       0.0345      0.004890   

C:\Users\natha\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:921: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan n

ValueError: Unknown label type: 'continuous-multioutput'

In [ ]:
# def recommendations(features):
# #     cursor = connection.cursor()
# #     cursor.execute("select * from songs")
# #     music = pd.DataFrame(cursor.fetchall())
# #     music.columns = [i[0] for i in cursor.description]


# #     cursor = connection.cursor()
# #     cursor.execute("select endTime, artistName, trackName, msPlayed from listening_history where user_id = %s", (session['id'],))
# #     user = pd.DataFrame(cursor.fetchall())
# #     user.columns = [i[0] for i in cursor.description]

#     # Read in tracks.csv
#     music = pd.read_csv("tracks.csv")
#     user = pd.read_csv("StreamingHistory.csv")

#     # Data Cleaning Steps
#     music = music.drop_duplicates(subset = ['name', 'artists'])
#     user_grouped = user.groupby(by = 'trackName').max().reset_index()
#     music['primary_artist'] = music['artists'].str.strip('[]').str.split(',').str[0].str.strip('\'\'')
#     merged = pd.merge(user_grouped, music, left_on = ['trackName', 'artistName'], right_on = ['name', 'primary_artist'])
#     merged.loc[merged['msPlayed'] <= 30000, 'Likes_Song'] = 0
#     merged.loc[merged['msPlayed'] > 30000, 'Likes_Song'] = 1

#     # Selecting subset of features
#     #filters = list(merged.columns[12:22])
#     filters = features
#     train = merged[filters + ["Likes_Song"]]
#     name = music['name']
#     artist = music['artists'].str.strip('[]').str.split(',').str[0].str.strip('\'\'')
#     test = music[filters]
#     user_x = train[filters]
#     user_y = train["Likes_Song"]

#     model = RandomForestClassifier()
#     model.fit(user_x, user_y)
#     predictions = model.predict(test)
#     test['recommendations'] = predictions
#     test['name'] = name
#     test['artist'] = artist

#     recommendations = test[test['recommendations'] == 1].sample(50)[['name','artist']]

In [11]:
tracks = pd.read_csv("tracks.csv")
tracks.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4


In [12]:
streamingHistory = pd.read_csv("StreamingHistory.csv")
streamingHistory.head()

,endTime,artistName,trackName,msPlayed
0,2021-11-20 21:51,Pitbull,Timber (feat. Ke$ha),204160
1,2021-11-20 21:55,Jason Derulo,Whatcha Say,221253
2,2021-11-20 21:59,Coleman Hell,Sitcom,2722
3,2021-11-20 21:59,Demi Lovato,Heart Attack,210840
4,2021-11-20 21:59,Capital Cities,Safe And Sound,137
